In [1]:
from trl import SFTTrainer

2025-04-25 11:41:47.904263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745581307.913777 1627182 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745581307.916663 1627182 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-25 11:41:47.927021: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, get_scheduler
from bitsandbytes.optim import Adam8bit,PagedAdam32bit
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
from peft import prepare_model_for_kbit_training
import torch
from IPython.display import  clear_output
import time
import gc,os
from torch.utils.data import Dataset, DataLoader
import numpy as np
from evaluation_utlis import rouge_bleu_score,get_prediction_per_sentence

device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"
TOKENIZER_PATH = DEFAULT_MODEL#"llama_odia_tokenizer"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=False,
    )


model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    quantization_config=bnb_config,
    use_safetensors=True,
    device_map=device,
)

print(model.get_memory_footprint()/(1024*1024)) 

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

def flush():
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2095.841064453125


In [2]:
from datasets import load_dataset

# Load the dataset
# dataset = load_dataset("OdiaGenAI/hardcode_odia_qa_105")
dataset = load_dataset('OdiaGenAI/odia_domain_context_train_v1')

In [3]:
res = rouge_bleu_score(model,tokenizer,dataset['train'],current_step=00,max_new_tokens=100,device='cuda')

/home/mohan.dash/miniconda3/envs/diffusion_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mohan.dash/miniconda3/envs/diffusion_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`

ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['ଗੁਜ\u200d্\u200cર,', 'ଅ\u200bস\u200cफ़\u200cग़\u200d\u200c\u200c\u200c\u200dक़\u200c', '\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c\u200c'],
Input references: [['2019', 'ମସିହାରେ', 'ଓଡ଼ିଶାରେ', '1.5', 'କୋଟିରୁ', 'ଅଧିକ', 'ପର୍ଯ୍ୟଟକଙ୍କୁ', 'ସ୍

In [ ]:
import evaluate
import numpy as np
from evaluation_utlis import rouge_bleu_score,get_prediction_per_sentence

# Load metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Store predictions and ground truths
pred_texts, true_texts = [], []
output_lines = []

# Collect examples
for idx in [11, 111, 222, 333, 444]:
    sample = dataset['train'][idx]
    question, pred_text, true_text = get_prediction_per_sentence(model, tokenizer, sample, max_new_tokens=100, device=device)
    pred_texts.append(pred_text)
    true_texts.append(true_text)

    # Format the result for the output file
    output_lines.append("=========================================")
    output_lines.append(f"Question:\n{question}\n")
    output_lines.append(f"Ground Truth:\n{true_text}\n")
    output_lines.append(f"Prediction:\n{pred_text}\n")

# Compute metrics
result_rouge = rouge.compute(
    predictions=pred_texts, 
    references=true_texts, 
    tokenizer=lambda x: x.split()
)

result_bleu = bleu.compute(
    predictions=pred_texts, 
    references=true_texts, 
)

# Calculate average ROUGE score
avg_rouge_score = np.mean(list(result_rouge.values()))
bleu_score = result_bleu['bleu']

# Add metrics to output
output_lines.append("=========================================")
output_lines.append(f"ROUGE Score (avg): {avg_rouge_score:.4f}")
output_lines.append(f"BLEU Score: {bleu_score:.4f}")

# Save to a text file
with open(f"evaluation_results_{0}.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(output_lines))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [11]:
# Compute metrics
result_rouge = rouge.compute(
    predictions=pred_texts, 
    references=true_texts, 
    tokenizer=lambda x: x.split()
)

result_bleu = bleu.compute(
    predictions=pred_texts, 
    references=true_texts, 
)

# Calculate average ROUGE score
avg_rouge_score = np.mean(list(result_rouge.values()))
bleu_score = result_bleu['bleu']

# Add metrics to output
output_lines.append("=========================================")
output_lines.append(f"ROUGE Score (avg): {avg_rouge_score:.4f}")
output_lines.append(f"BLEU Score: {bleu_score:.4f}")

# Save to a text file
with open(f"evaluation_results_{0}.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(output_lines))

In [10]:
result_bleu

{'bleu': 0.0,
 'precisions': [0.12698412698412698, 0.017241379310344827, 0.0, 0.0],
 'brevity_penalty': 7.665540221071256e-05,
 'length_ratio': 0.09545454545454546,
 'translation_length': 63,
 'reference_length': 660}